# Lab 1

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

### Getting some constants

We get some constants from the local execution environment.

In [ ]:
import boto3
import sagemaker
import time
import json
import base64
import requests
import argparse
import os

from boto3.dynamodb.conditions import Key
from pipeline import get_pipeline

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()
account_number = boto3.client('sts').get_caller_identity().get('Account')
client = boto3.client('sagemaker')

In [3]:
model_package_group_name = f"CustomerChurnGenericModel"
pipeline_name = f"CustomerChurnPipeline"
generic_model_data_bucket_name = f"sagemaker-mlaas-pooled-{region}-{account_number}"

## Upload Sample Training Data

In [ ]:
s3_bucket_prefix = "generic"
local_path = "data/AnyCompany"
sagemaker.Session().upload_data(path=local_path,bucket=generic_model_data_bucket_name,key_prefix=s3_bucket_prefix)

### Get the pipeline instance
Here we get the pipeline instance from your pipeline module so that we can work with it.

In [ ]:
pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

### Submit the pipeline to SageMaker and start execution
Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=role)

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [7]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ProcessingInstanceCount=1,
        TrainDataPath = f"s3://{generic_model_data_bucket_name}/generic/train/train.csv",
        TestDataPath = f"s3://{generic_model_data_bucket_name}/generic/test/test.csv",
        ValidationDataPath = f"s3://{generic_model_data_bucket_name}/generic/validation/validation.csv",
        ModelPath = f"s3://{generic_model_data_bucket_name}/model_artifacts",
        BucketName = f"{generic_model_data_bucket_name}",
        ObjectKey = f"model_artifacts/basic_tier/output",
        ModelPackageGroupName = f"{model_package_group_name}",
        ModelVersion = "1"
    )
)


### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [ ]:
execution.describe()

We can wait for the execution by invoking `wait()` on the execution. The pipeline execution can take about 10 minutes to be completed. For the purpose of this workshop, a trained model artifact has already been provided in the S3 folder to allow us to proceed with the deployment without waiting for this pipeline to be completed. We will back and check when its executive is completed. 

In [9]:
# execution.wait()

## Create / Update Basic Tier SageMaker Endpoint

We will deploy the model to a SageMaker dedicated endpoint.

In [ ]:
timestamp = int(time.time())

model_name=f'GenericModel-{timestamp}'
endpoint_config_name = f"EndpointConfig-{model_name}-{timestamp}"
endpoint_name = "Endpoint-GenericModel"
shared_inference_api_url = "https://lmla2luuak.execute-api.us-east-1.amazonaws.com/" # Example: https://3289dfakjkak.execute-api.us-east-1.amazonaws.com/
model_artifact_file_name = "generic.model.1.tar.gz"

# get image URI
image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=region,
        version="1.0-1",
        py_version="py3",
        instance_type='ml.t2.medium',
    )

# create sagemaker model
create_model_api_response = client.create_model(
                                    ModelName=model_name,
                                    PrimaryContainer={
                                        'Image': image_uri,
                                        'ModelDataUrl': f"s3://{generic_model_data_bucket_name}/model_artifacts/basic_tier/output/{model_artifact_file_name}",
                                        'Environment': {}
                                    },
                                    ExecutionRoleArn=role
                            )
print ("create_model API response", create_model_api_response)

# create sagemaker endpoint config
update_endpoint_config_api_response = client.create_endpoint_config(
                                            EndpointConfigName=endpoint_config_name,
                                            ProductionVariants=[
                                                {
                                                    'VariantName': 'prod1',
                                                    'ModelName': model_name,
                                                    'InitialInstanceCount': 1,
                                                    'InstanceType': 'ml.t2.medium'
                                                },
                                            ]
                                       )

print ("update_endpoint_config API response", update_endpoint_config_api_response)

# update sagemaker endpoint
update_endpoint_api_response = client.update_endpoint(
                                    EndpointName=endpoint_name,
                                    EndpointConfigName=endpoint_config_name,
                                )

print ("update_endpoint API response", update_endpoint_api_response)    

print(f"Updating endpoint {endpoint_name}...")

In [11]:
#waiter = client.get_waiter('endpoint_in_service')
#waiter.wait(EndpointName=endpoint_name)
#print(f"Endpoint {endpoint_name} is in service.")

## Onboard Basic Tier Tenants

The following Script retrieves the SaaS Control Plane URL and the username and password you require to login.

In [ ]:
!chmod +x ../setup/create-admin-user.sh
!../setup/create-admin-user.sh

<div class="alert alert-block alert-info">
<font color='black'><b>!!! Follow instructions in the Workshop Tutorial to onboard 2 (two) Basic Tier Tenants</b></font>
</div>

## Lab1 Test Inference

Execute the cell below to initialize the inference helper functions

In [15]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
def basic_tier_run_inference(username, password, tenant_name, request):
    api_url=get_api_url(tenant_name)
    jwt = get_jwt(username, password, tenant_name, api_url)
    invoke_inference_endpoint(jwt, request, api_url+'basic_inference')
    
def run_inference(username, password, tenant_name, request):
    api_url=get_api_url(tenant_name)
    jwt = get_jwt(username, password, tenant_name, api_url)
    invoke_inference_endpoint(jwt, request, api_url+'inference')

def get_jwt(username, password, tenant_name, api_url):

    auth_str = f'{username}:{password}'
    byte_str = auth_str.encode('ascii')
    auth_b64 = base64.b64encode(byte_str)

    headers = {
        'Authorization': 'Basic {}'.format(auth_b64.decode('ascii')),
        'tenant-name': tenant_name
    }

    try:
        print("Getting JWT")
        response = requests.get(api_url + 'jwt', headers=headers)
        # print(response.text if response.text else response.reason)
        jwt = json.loads(response.text)['jwt']
        return jwt
    except Exception as e:
        print("Error getting JWT", e)
        
        
    
def invoke_inference_endpoint(jwt, request, api_url):

    headers = {
        'Authorization': 'Bearer {}'.format(jwt),
        'Content-Type': 'text/csv'
    }
    
    try:
        print(f"Inference request with: {request}")
        response = requests.post(api_url, headers=headers, data=request)
        print(response.text if response.text else response.reason)
    except Exception as e:
        print("Error executing inference request", e)

def get_api_url(tenant_name):
    dynamodb = boto3.resource('dynamodb')
    table_tenant_details = dynamodb.Table('MLaaS-TenantDetails')
    
    try:
        
        tenant_details = table_tenant_details.query(
            IndexName='tenantName-index',
            KeyConditionExpression=Key('tenantName').eq(tenant_name)
        )

        api_url = tenant_details['Items'][0]['apiGatewayUrl']
        print(api_url)
        return api_url      
    except Exception as error:
        print(f'[Error]: {error}')
        
    


First let us send an inference request for our first tenant (basic1).

In [ ]:
basic_tenant_1_username = "basic1@example.com"# Example: basic1@example.com
basic_tenant_1_name = "basic1" # Example: basic1
basic_tenant_1_request = "84,1,3,98,2,4" 

basic_tier_run_inference(username=basic_tenant_1_username,password="Mlaa$1234",tenant_name=basic_tenant_1_name,request=basic_tenant_1_request)

Next we send an inference request for our second tenant (basic2).

In [ ]:
basic_tenant_2_username = "basic2@example.com" # Example: basic2@example.com
basic_tenant_2_name = "basic2" 
basic_tenant_2_request = "1,0,3,105,2,9"

basic_tier_run_inference(username=basic_tenant_2_username,password="Mlaa$1234",tenant_name=basic_tenant_2_name,request=basic_tenant_2_request)

# Lab 2

## Onboard Advanced Tier Tenants

<div class="alert alert-block alert-info">
<font color='black'><b>!!! Follow instructions in the Workshop Studio to onboard 2 (two) Advanced Tier Tenants</b></font>
</div>

## Upload Training Data

Execute the cell below to initialize the file upload helper function

In [14]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

def upload_file(username, password, tenant_name, file, file_type):
    api_url=get_api_url(tenant_name)
    jwt = get_jwt(username, password, tenant_name, api_url)
    file_name =  os.path.basename(file)
    
    headers = {
        'Authorization': 'Bearer {}'.format(jwt),
        'Content-Type': 'text/csv',
        'file-name': file_name,
        'file-type': file_type
    }

    try:
        print(f"Uploading file: {file}")
        response = requests.put(
            api_url + 'upload', headers=headers, data=open(file, 'rb'))
        print(response.text if response.text else response.reason)
    except Exception as e:
        print("Error uploading file", e)

Run the python function above to upload your advanced tenant training data. This function will handle both, getting a JWT Token and uploading the training data to the correct S3 tenant prefix.

NOTE: Provide the following data before executing the upload_file python function
> First, upload the data for advanced tenant 1

In [21]:
advanced_tenant_1_username = "advanced1@example.com" #eg. "advanced6@example.com" 
advanced_tenant_1_name = "advanced1" 
advanced_tenant_1_file = "data/Advanced-Tenant1/advanced1_dataset.csv" 

In [ ]:
upload_file(username=advanced_tenant_1_username, password="Mlaa$1234",tenant_name=advanced_tenant_1_name,file_type="csv",file=advanced_tenant_1_file)

NOTE: Provide the following data before executing the upload_file python function

In [25]:
advanced_tenant_2_username = "advanced2@example.com" #e.g., "advanced2@example.com" 
advanced_tenant_2_name = "advanced2" 
advanced_tenant_2_file = "data/Advanced-Tenant2/advanced2_dataset.csv" 

In [ ]:
upload_file(username=advanced_tenant_2_username, password="Mlaa$1234",tenant_name=advanced_tenant_2_name,file_type="csv",file=advanced_tenant_2_file)

<div class="alert alert-block alert-info">
<font color='black'><b>!!! Verify that execution of 2 new training pipelines has started. Use SageMaker console to check with execution status. Wait for the execution of the pipelines to complete</b></font>
</div>

## Lab2 Test Inference

In [ ]:
advanced_tenant_100_username = "advanced100@example.com" 
advanced_tenant_100_name = "advanced100"
advanced100_request = "18,0,3,105,2,9" 
run_inference(username=advanced_tenant_100_username,password="Mlaa$1234",tenant_name=advanced_tenant_100_name,request=advanced100_request)

In [ ]:
advanced_tenant_200_username = "advanced200@example.com" 
advanced_tenant_200_name = "advanced200"
advanced200_request = "18,0,3,105,2,9" 
run_inference(username=advanced_tenant_200_username,password="Mlaa$1234",tenant_name=advanced_tenant_200_name,request=advanced200_request)

# Lab 3

<div class="alert alert-block alert-info">
<font color='black'><b>!!! Follow instructions in the Workshop Studio to onboard 1 Premium Tier Tenant named Premium1</b></font>
</div>

## Upload Training Data

## Lab3 Test Inference

In [45]:
premium_tenant_100_username = "premium100@example.com" 
premium_tenant_100_name = "premium100" 
premium100_request = "18,0,3,105,2,9" 
run_inference(username=premium_tenant_100_username,password="Mlaa$1234",tenant_name=premium_tenant_100_name,request=premium100_request)

# Lab4

## Test Inference

We have hardcoded the tenant **advanced1** tenant id. Now lets run inference request for different tenant **advanced2**. We will be using **advanced2** tenant users username and tenant name as shown below. If you used different tenant name and username please make sure to use the correct tenant name and username. 

In [ ]:
advanced_tenant_2_username="test+advanced2@example.com"
advanced_tenant_2_name = "advanced2"
advanced_tenant_2_request = "18,0,3,105,2,9" 
run_inference(username=advanced_tenant_2_username,password="Mlaa$1234",tenant_name=advanced_tenant_2_name,request=advanced_tenant_2_request)